<a href="https://colab.research.google.com/github/Arjun-08/DS202-2025-HW3-ARJUN-/blob/main/DS202_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`pydivsufsort` is a Python binding for the **DivSufSort** algorithm, used for efficient **suffix array construction**. You likely used it in your **X chromosome data analysis** for **BWT-based read alignment**.

🔗 [PyDivSufSort on PyPI](https://pypi.org/project/pydivsufsort/)

In [16]:
!pip install pydivsufsort

In [17]:
import pydivsufsort as divsufsort

In [18]:
#!/usr/bin/env python3
import time
import resource
import pydivsufsort as divsufsort

Computes the suffix array using pydivsufsort.The suffix array is an array of starting indices of sorted suffixes of the input text.
    


In [19]:
def compute_suffix_array(text):

    return divsufsort.divsufsort(text)

Computes the Longest Common Prefix (LCP) array using the suffix array. The LCP array stores the length of the longest common prefix betweeN adjacent suffixes in SA.
    


In [20]:
def compute_lcp(text, SA):

    n = len(text)
    rank = [0] * n  ## Rank array to store the index of each suffix in SA
    for i, si in enumerate(SA):
        rank[si] = i

    lcp = [0] * n
    h = 0

    for i in range(n):
        if rank[i] > 0:
            j = SA[rank[i] - 1]
            while i + h < n and j + h < n and text[i + h] == text[j + h]:
                h += 1
            lcp[rank[i]] = h  # Store the LCP length
            if h > 0:
                h -= 1
    return lcp


    Finds the longest repeated substring in the text using SA and LCP.
    

In [21]:
def longest_repeat(text, SA, lcp):

    n = len(text)
    max_lcp = max(lcp)  # Find the longest LCP value
    if max_lcp == 0:
        return (0, [], "")

    candidate = None
    positions = set()

    for i in range(1, n):
        if lcp[i] == max_lcp:
            current = text[SA[i]: SA[i] + max_lcp]
            if candidate is None:
                candidate = current
                positions.update([SA[i], SA[i - 1]])
            else:
                if current == candidate:
                    positions.update([SA[i], SA[i - 1]])
                elif current < candidate:
                    candidate = current
                    positions = {SA[i], SA[i - 1]}

    return (max_lcp, sorted(positions), candidate)


    Loads a DNA sequence from a FASTA file.


In [22]:
def load_fasta(filename):

    seq_lines = []
    with open(filename, 'r') as f:
        for line in f:
            if line.startswith('>'):  # Ignore FASTA headers
                continue
            seq_lines.append(line.strip())
    return "".join(seq_lines)

In [23]:
def main():
    input_file = "/content/sequence.fasta"  # Path to FASTA file
    text = load_fasta(input_file)
    n = len(text)
    print("Input text length: {} characters".format(n))

    start_time = time.time()  # Start timing for SA and LCP computation

    SA = compute_suffix_array(text)  # Compute suffix array
    LCP = compute_lcp(text, SA)  # Compute LCP array

    end_time = time.time()
    wall_clock_time = end_time - start_time  # Compute total time taken

    max_repeat_length, positions, repeat_substring = longest_repeat(text, SA, LCP)  # Find longest repeat

    usage = resource.getrusage(resource.RUSAGE_SELF)  # Get memory usage
    peak_memory_kb = usage.ru_maxrss  # Peak memory usage in KB

    print("\nRESULTS:")
    print("---------")
    print("Length of the longest repeat: {}".format(max_repeat_length))
    print("Longest repeated substring: '{}'".format(repeat_substring))
    print("Starting positions of the longest repeat: {}".format(positions))
    print("Wall-clock time for SA and LCP construction: {:.6f} seconds".format(wall_clock_time))
    print("Peak memory usage: {} KB".format(peak_memory_kb))

if __name__ == '__main__':
    main()


Input text length: 62460029 characters

RESULTS:
---------
Length of the longest repeat: 81262
Longest repeated substring: '...'
Starting positions of the longest repeat: [58413702, 59261387]
Wall-clock time for SA and LCP construction: 701.606730 seconds
Peak memory usage: 4303592 KB
